# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [1]:
from utils import get_dataset
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
dataset = get_dataset("/home/workspace/data/*/*.tfrecord")

INFO:tensorflow:Reading unweighted datasets: ['/home/workspace/data/*/*.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['/home/workspace/data/*/*.tfrecord']
INFO:tensorflow:Number of filenames to read: 99
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
Instructions for updating:
Use `tf.data.Dataset.map()


## Write a function to display an image and the bounding boxes

Implement the `display_images` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [ ]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    """
    # ADD CODE HERE
    try:
        #color map
        color_map = {1:'red',2:'blue',4:'green'}
        fig, ax = plt.subplots()
        im = batch["image"].numpy()
        ax.imshow(im)
        #draw bounding boxes and add color to different class labels
        coords = batch["groundtruth_boxes"].numpy()
        labels = batch['groundtruth_classes'].numpy()
        for i in range(len(coords)):
            xy = (coords[i][1]*im.shape[1], coords[i][0]*im.shape[0])
            #taken from class notes
            width = (coords[i][3] - coords[i][1]) * im.shape[1]
            height = (coords[i][2] - coords[i][0]) * im.shape[0]
            rec = patches.Rectangle(xy, width, height, linewidth=1, edgecolor=color_map[labels[i]], facecolor='none')
            ax.add_patch(rec)
        plt.axis("off")
    except Exception as err:
        print(err)
    pass

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
## STUDENT SOLUTION HERE
plt.figure(figsize=(20,15))
for i in range(10):
    for batch in dataset.shuffle(10,reshuffle_each_iteration=True).take(1):
        display_instances(batch)

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...</br>
*It would be interesting to sample 100 images in the set and see the distribution of labels this would provide an representation of the amount of examples that we have and could hypothesize about some sparse examples such as bikes.*

In [ ]:
#taken from label_map.pbtxt
label_map = {1:'vehicle', 2:'pedestrian', 4:'cyclist'}

df_dict = {'vehicle':0,'pedestrian':0,'cyclist':0}
for batch in dataset.take(100):
    classes = batch['groundtruth_classes'].numpy()
    if classes.size == 0:
        pass
    else:
        labels = np.vectorize(label_map.get)(classes)
        for key in df_dict.keys():
            df_dict[key] = df_dict[key]+(labels == key).sum()
df = pd.DataFrame(list(df_dict.items()),columns=['label','qty'])
df

In [6]:
plt.figure(figsize = (15,8))
sns.histplot(df, x='qty',stat='percent',hue='label')

NameError: name 'sns' is not defined

<Figure size 1080x576 with 0 Axes>